In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/salary_analysis.db")
df = pd.read_sql_query("SELECT * FROM salary_data;", conn)
conn.close()

df.head()


,Country,work_year,experience_level,employment_type,job_title,salary_in_usd,remote_ratio,company_size,Cost of Living Index,Rent Index,...,country_India,country_Japan,country_Netherlands,country_Other,country_Spain,country_United Kingdom,country_United States,exp_x_role,exp_x_remote,size_x_gdp
0,Germany,2020,Mid,FT,Data Scientist,79833,0,L,62.200000,24.400000,...,0,0,0,0,0,0,0,8,0.0,142139.295584
1,Japan,2020,Senior,FT,Machine Learning Scientist,260000,0,S,46.100000,13.400000,...,0,1,0,0,0,0,0,12,0.0,40028.734173
2,United Kingdom,2020,Senior,FT,Big Data Engineer,109024,50,M,62.000000,31.100000,...,0,0,0,0,0,1,0,6,1.5,80809.612448
3,Honduras,2020,Mid,FT,Product Data Analyst,20000,0,S,63.606218,34.282185,...,0,0,0,1,0,0,0,6,0.0,2307.614943
4,United States,2020,Senior,FT,Machine Learning Engineer,150000,50,L,70.400000,41.700000,...,0,0,0,0,0,0,1,12,1.5,193204.522306


In [2]:
import plotly.express as px

fig = px.histogram(df, x="salary_in_usd", nbins=50, 
                   title="Salary Distribution (USD)",
                   color_discrete_sequence=["#636EFA"])
fig.update_layout(bargap=0.1)
fig.show()


In [3]:
fig = px.box(df, x="experience_level", y="salary_in_usd",
             color="experience_level",
             title="Salary by Experience Level")
fig.show()


In [4]:
remote_avg = df.groupby("remote_ratio")["salary_in_usd"].mean().reset_index()
fig = px.bar(remote_avg, x="remote_ratio", y="salary_in_usd",
             color="remote_ratio", title="Average Salary by Remote Work Type")
fig.show()


In [5]:
import plotly.express as px

fig = px.scatter(
    df,
    x="Cost of Living Index",
    y="salary_in_usd",
    color="experience_level",
    size="salary_in_usd",
    hover_data=["Country", "employment_type"],
    title="Salary vs Cost of Living by Experience"
)
fig.show()


In [6]:
country_avg = (
    df.groupby("Country")["salary_in_usd"]
      .mean()
      .sort_values(ascending=False)
      .head(10)
      .reset_index()
)

fig = px.bar(
    country_avg,
    x="salary_in_usd",
    y="Country",
    orientation="h",
    color="salary_in_usd",
    color_continuous_scale="Viridis",
    title="Top 10 Countries by Average Salary"
)
fig.show()


In [7]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")


In [8]:
import numpy as np

corr = df.corr(numeric_only=True)
fig = px.imshow(corr, text_auto=True, aspect="auto",
                title="Feature Correlation Heatmap",
                color_continuous_scale="RdBu_r")
fig.show()


In [9]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Load data
conn = sqlite3.connect("../data/salary_analysis.db")
df = pd.read_sql_query("SELECT * FROM salary_data;", conn)
conn.close()
df.columns = df.columns.str.lower().str.strip().str.replace(" ", "_")

# Check your column names (just to confirm)
print(df.columns.tolist())

# Build app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("🌍 Global Salary Dashboard", style={"textAlign": "center"}),
    
    html.Label("Select a Country:", style={"fontWeight": "bold"}),
    dcc.Dropdown(
        options=[{"label": c, "value": c} for c in df["country"].unique()],
        value="United States",
        id="country-filter"
    ),
    
    dcc.Graph(id="salary-dist")
])

@app.callback(
    Output("salary-dist", "figure"),
    Input("country-filter", "value")
)
def update_chart(country):
    dff = df[df["country"] == country]
    fig = px.histogram(
        dff, 
        x="salary_in_usd",
        nbins=40,
        title=f"Salary Distribution in {country}",
        color_discrete_sequence=["#7d0be1ff"]
    )
    fig.update_layout(
        xaxis_title="Salary (USD)",
        yaxis_title="Frequency",
        template="plotly_white"
    )
    return fig

if __name__ == "__main__":
    app.run(debug=True)


['country', 'work_year', 'experience_level', 'employment_type', 'job_title', 'salary_in_usd', 'remote_ratio', 'company_size', 'cost_of_living_index', 'rent_index', 'local_purchasing_power_index', 'cost_of_living_plus_rent_index', 'gdp', 'log_salary', 'experience_numeric', 'salary_k', 'adjusted_salary', 'rent_burden_ratio', 'gdp_to_power_ratio', 'role_category', 'role_numeric', 'remote_onsite', 'remote_remote', 'company_size_num', 'country_france', 'country_germany', 'country_greece', 'country_india', 'country_japan', 'country_netherlands', 'country_other', 'country_spain', 'country_united_kingdom', 'country_united_states', 'exp_x_role', 'exp_x_remote', 'size_x_gdp']
